In [1]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import pandas as pd
import numpy as np
from flask_app.function_api import load_dataset_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
pd.set_option("display.max_columns", None)
import sqlite3

In [12]:
df_documents = load_dataset_similarity()

5153
6
https://www.datagrandest.fr/data4citizen/d4c/api/datasets/2.0/search/start=1&rows=1000
https://www.datagrandest.fr/data4citizen/d4c/api/datasets/2.0/search/start=1000&rows=1000
https://www.datagrandest.fr/data4citizen/d4c/api/datasets/2.0/search/start=2000&rows=1000
https://www.datagrandest.fr/data4citizen/d4c/api/datasets/2.0/search/start=3000&rows=1000
https://www.datagrandest.fr/data4citizen/d4c/api/datasets/2.0/search/start=4000&rows=1000
https://www.datagrandest.fr/data4citizen/d4c/api/datasets/2.0/search/start=5000&rows=153


/home/dimitri/Documents/code/python/projet_ia_datagrand/flask_app/function_api.py:208: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dataset_merge['documents'] = df_dataset_merge['documents'].str.replace(


## Test Doc2vec

In [23]:
# Load doc2vec model
model = Doc2Vec.load("w2d.model")

In [10]:
def most_similar(doc_id,similarity_matrix,matrix):
    for i in range(len(dataframe)):
        
        if matrix=='Cosine Similarity':
            similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
        for ix in similar_ix:
            if ix==doc_id:
                continue
            print (f'Document: {df_documents.iloc[doc_id]["documents"]}')
            print ('\n')
            print ('Similar Documents:')
            print('\n')
            print (f'Document: {df_documents.iloc[ix]["documents"]}')
            print (f'{matrix} : {similarity_matrix[doc_id][ix]}')
                    

In [19]:
def most_similar(dataframe,similarity_matrix,matrix):
    for i in range(len(dataframe)):
        print(i)
        with open('similarity_document.txt','a') as w:
            w.write(f'Document sélectionner :')
            w.write("\n")
            w.write(f"Document: {df_documents.iloc[i]['id']} {df_documents.iloc[i]['documents']}\n")
            w.write(f"url fiche sélectionner: {df_documents.iloc[i]['url']};\n")
            w.write("\n")
            w.write("Similar Documents:")
            if matrix=="Cosine Similarity":
                similar_ix=np.argsort(similarity_matrix[i])[::-1]
            for ix in similar_ix:
                if ix == i:
                    continue
                elif similarity_matrix[i][ix]>= 0.90:   
                    w.write("\n")
                    w.write(str(f"Document: {df_documents.iloc[ix]['id']} {df_documents.iloc[ix]['documents']}\n"))
                    w.write(str(f"url de la fiche: {df_documents.iloc[ix]['url']}\n"))
                    w.write(str(f"{matrix} : {similarity_matrix[i][ix]};\n"))
                    w.write("\n")
        

In [ ]:
def most_similar(data,similarity_matrix,matrix):
    df_document_select = pd.DataFrame(columns = ['id_fiche_select','documents_select','url_fiche_select','id_fiche_similar','doc_fiche_similar','url_document_similair','cosine similarity'])
    # df_document_similarity = pd.DataFrame(columns = ['id','','documents','url','cosine similarity'])
    for i in range(len(data)):
            # df_documents_select_columns = df_document_select.columns[i]
            df_document_select[['id_fiche_select','documents_select','url_fiche_select']] = [df_documents.iloc[i]['id'],df_documents.iloc[i]['documents'],df_documents.iloc[i]['url']]
            print(df_document_select)
            if matrix=="Cosine Similarity":
                similar_ix=np.argsort(similarity_matrix[i])[::-1]
            for ix in similar_ix:
                if ix == i:
                    continue 
                elif similarity_matrix[i][ix]>= 0.95:
                    # df_documents_similar_columns = df_document_similarity.columns[i]
                    df_document_select[['id_fiche_similar','doc_fiche_similar','url_fiche_similar','cosine similarity']] = [df_documents.iloc[ix]['id'],df_documents.iloc[ix]['documents'],df_documents.iloc[ix]['url'],similarity_matrix[i][ix]]
    df_document_select.to_csv('df_documents_select.csv')

In [32]:
# Test 2: Cosine Similarity
def most_similar(data,similarity_matrix,matrix):
    global df_merge_similar
    # Create two list for append informations about documents we want to compare similarity and another list to append the value of similarity
    results_fiche_select = []
    results_fiche_similar = []
    for i in range(len(data)):
            print(i)
        # Create a dictionnary with information about the document we want to compare similarity
            dict_fiche_select = dict(id_fiche_select=data.iloc[i]['id'],url_fiche_select=data.iloc[i]['url'])
            # Append the dictionnary to the list
            results_fiche_select.append(dict_fiche_select)
            # Test the similarity of documents 
            if matrix=="Cosine Similarity":
                # Order results of cosine similarity in descending order
                similar_ix=np.argsort(similarity_matrix[i])[::-1]
            for ix in similar_ix:
                if ix == i:
                    continue 
                elif similarity_matrix[i][ix]>= 0.95:
                    # df_documents_similar_columns = df_document_similarity.columns[i]
                    dict_fiche_similar = dict(id_fiche_select=data.iloc[i]['id'],id_fiche_similar=data.iloc[ix]['id'],url_document_similair=data.iloc[ix]['url'],cosine_similarity=similarity_matrix[i][ix])
                    results_fiche_similar.append(dict_fiche_similar)
    df_fiche_select = pd.DataFrame(results_fiche_select)
    df_fiche_similar = pd.DataFrame(results_fiche_similar)
    df_fiche_select.to_csv('fiche_select.csv',index=False)
    df_fiche_similar.to_csv('fiche_similar.csv',index=False)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = df_documents['documents'].tolist()
document_embeddings = model.encode(sentences,show_progress_bar=True)

pairwise_similarities=cosine_similarity(document_embeddings)
                                                         
df_similar = most_similar(df_documents,pairwise_similarities,'Cosine Similarity')

In [ ]:
display(df_similar)

In [8]:
df_fiche_select = pd.read_csv('fiche_select.csv')
df_fiche_similar = pd.read_csv('fiche_similar.csv')

In [29]:
df_merge_similar = pd.merge(df_fiche_select,df_fiche_similar,on='id_fiche_select')
df_merge_similar.drop(['documents_select','doc_fiche_similar'],axis=1,inplace=True)

In [ ]:
display(df_merge_similar)

In [ ]:
df_merge_similar.set_index(['id_fiche_select','url_fiche_select'])

In [ ]:
display(df_merge_similar.head())

In [24]:
df_groupby = df_merge_similar.groupby(['id_fiche_select'])['id_fiche_similar','url_document_similair','cosine_similarity']
df_groupby.first()

/tmp/ipykernel_6879/1183482337.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_groupby = df_merge_similar.groupby(['id_fiche_select'])['id_fiche_similar','url_document_similair','cosine_similarity']


,id_fiche_similar,url_document_similair,cosine_similarity
id_fiche_select,,,
0,3005,https://grandestprod-backoffice.data4citizen.com//dataset/zone-dalea-du-ppri-madon-amont,0.999256
1,554,https://grandestprod-backoffice.data4citizen.com//dataset/zones-reglementees-surfaces-du-ppri-moselle-centre,0.994170
2,1,https://grandestprod-backoffice.data4citizen.com//dataset/zones-reglementees-surfaces-du-ppri-madon-centre,0.990865
3,8,https://grandestprod-backoffice.data4citizen.com//dataset/perimetres-du-ppri-madon-amont,0.990452
4,2758,https://grandestprod-backoffice.data4citizen.com//dataset/perimetres-du-ppri-meuse,0.997781
...,...,...,...
5145,3432,https://grandestprod-backoffice.data4citizen.com//dataset/carte-ile-du-rohrschollen-en-1872,0.990914
5146,5141,https://grandestprod-backoffice.data4citizen.com//dataset/carte-ile-du-rohrschollen-en-1778,0.977001
5150,5152,https://grandestprod-backoffice.data4citizen.com//dataset/donnees-carte-surfacique-1991-1992-des-teneurs-en-nitrates-des-couches-superficelles-de-la-napp,0.973213


In [13]:
df_merge_similar.groupby(by='id_fiche_select')

## Test import des données en texte

In [ ]:
df_cosine_read_csv = pd.read_csv('similarity_document.txt',sep=';',engine='python')

In [ ]:
df_cosine = pd.read_fwf('similarity_document.txt',header = None)

In [ ]:
display(df_cosine)

,0,1,2,3,4
0,Document sélectionner :,NaN,NaN,NaN,NaN
1,"Document: 0 Table contenant les assiettes surfaciques liées aux servitudes de la catégorie AC1 (Monuments historiques) dans le Haut-RhinLes servitudes de catégorie AC1 concernent les servitudes de protection des monuments historiques classés ou inscrits :- Classement au titre des monuments historiques : ces servitudes concernent les immeubles ou les parties d'immeubles dont la conservation présente du point de vue de l'histoire ou de l'art un intérêt public. Les propriétaires d'immeubles classés ne peuvent effectuer de travaux de restauration, de réparation ou de modification sans autorisation préalable du préfet de région ou du ministre chargé de la culture. - Inscription au titre des monuments historiques : ces servitudes concernent les immeubles ou parties d'immeubles qui, sans justifier une demande de classement immédiat, présentent un intérêt d'histoire ou d'art suffisant pour en rendre désirable la préservation. Les propriétaires d'immeubles inscrits ne peuvent procéder à aucune modification sans déclaration préalable ; aucune autorisation d'urbanisme ne peut être délivrée sans accord préalable du préfet de région. - Immeubles adossés aux immeubles classés et immeubles situés dans le champ de visibilité des immeubles classés ou inscrits : Tout immeuble en contact avec un immeuble classé, en élévation, au sol ou en sous-sol est considéré comme immeuble adossé. Toute partie non protégée au titre des monuments historiques d'un immeuble partiellement classé est considérée comme immeuble adossé. Est considéré comme étant situé dans le champ de visibilité d'un immeuble classé ou inscrit, tout autre immeuble, nu ou bâti, visible du premier ou visible en même temps que lui est situé dans un périmètre déterminé par une distance de 500m du monument. Ce périmètre de 500m peut être modifié ou adapté : • le périmètre de protection adapté (PPA) : lorsqu'un immeuble non protégé fait l'objet d'une procédure d'inscription, de classement, ou d'instance de classement, l'architecte des bâtiments de France (ABF) peut proposer un périmètre de protection adapté en fonction de la nature de l'immeuble et de son environnement. • Le périmètre de protection modifié (PPM) : le périmètre institué autour d'un monument historique peut être modifié sur proposition de l'ABF. Cette ressource décrit les assiettes des servitudes de la catégorie AC1, à savoir tout ou partie d'un immeuble, ou un ou des périmètres définis autour du monument : • soit le rayon de 500 mètres fixé par la loi, • soit un périmètre étendu au-delà des 500 mètres ou au contraire réduit (bâtiments industriels, édicules ruraux, ...) ou encore spécifique (cône de vue, perspective","monumentale, ...), • soit un périmètre limité à des secteurs géographiques les plus sensibles ou étendu à des éléments de paysage situés au-delà des 500 mètres mais entretenant avec le monument une relation forte (perspective d'unevoie,",paysage,"écrin,",...).
2,url fiche sélectionner: https://grandestprod-backoffice.data4citizen.com//dataset/table-contenant-les-assiettes-surfaciques-liees-aux-servitudes-de-la-categorie-ac1-monuments-hi;,NaN,NaN,NaN,NaN
3,Similar Documents:,NaN,NaN,NaN,NaN
4,"Document: 2611 SUP AC1 - Servitudes relatives aux monuments historiques en Meurthe-et-MoselleLes servitudes de catégorie AC1 concernent les servitudes de protection des monuments historiques classés ou inscrits : - Classement au titre des monuments historiques : ces servitudes concernent les immeubles ou les parties d'immeubles dont la conservation présente du point de vue de l'histoire ou de l'art un intérêt public. Les propriétaires d'immeubles classés ne peuvent effectuer de travaux de restauration, de réparation ou de modification sans autorisation préalable du préfet de région ou du ministre chargé de la culture. - Inscription au titre des monuments historiques : ces servitudes concernent les immeubles ou parties d'immeubles qui, sans justifier une demande de cla